In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import tensorflow as tf
import numpy as np
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
data_dir = '/content/drive/My Drive/Colab Notebooks/data'
train_data_dir = '/content/drive/My Drive/Colab Notebooks/data/train'
test_data_dir = '/content/drive/My Drive/Colab Notebooks/data/test' 
model_path = '/content/drive/My Drive/Colab Notebooks/data/SGS_BasicTransferLearningModel.h5'

In [ ]:
NUM_CLASSES = 10
CHANNELS = 3
IMG_HEIGHT = 224
IMG_WIDTH = 224

aug_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,                                                                      
                                                                      samplewise_center=True, 
                                                                      samplewise_std_normalization=True,
                                                                      rotation_range=60,
                                                                      width_shift_range=0.2,
                                                                      height_shift_range=0.2,
                                                                      shear_range=0.2,
                                                                      zoom_range=0.2,
                                                                      horizontal_flip=True,
                                                                      vertical_flip=True,
                                                                      fill_mode='nearest',
                                                                      validation_split = 0.2)
norm_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,                                                                      
                                                               samplewise_center=False, 
                                                               samplewise_std_normalization=False,
                                                               validation_split = 0.2)


aug_train_generator = aug_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (224,224),
                                                    batch_size = 64,
                                                    class_mode = 'binary',
                                                    shuffle=True,
                                                    subset = 'training')

aug_validation_generator = aug_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (224,224),
                                                    batch_size = 64,
                                                    class_mode = 'binary',
                                                    shuffle=True,
                                                    subset = 'validation')

norm_train_generator = norm_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (224,224),
                                                    batch_size = 64,
                                                    class_mode = 'binary',
                                                    shuffle=True,
                                                    subset = 'training')

norm_validation_generator = norm_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (224,224),
                                                    batch_size = 64,
                                                    class_mode = 'binary',
                                                    shuffle=True,
                                                    subset = 'validation')

In [ ]:

base_model_VGC16 = tf.keras.applications.VGG16(input_shape=(224,224,3),
                                               include_top=False,
                                               weights=None,
                                               classes = 10
                                    )
base_model_VGC16.trainable = True

Layer1 = tf.keras.layers.Flatten()
Layer2 = tf.keras.layers.Dense(1024,activation='relu')
Layer3 = tf.keras.layers.Dropout(.6)
Layer4 = tf.keras.layers.Dense(128,activation='relu')
Layer5 = tf.keras.layers.Dropout(.5)
Layer6 = tf.keras.layers.Dense(64,activation='relu')
Layer7 = tf.keras.layers.Dropout(.5)
Layer8 = tf.keras.layers.Dense(8,activation='relu')
Layer9 = tf.keras.layers.Dropout(.2)
prediction_layer = tf.keras.layers.Dense(1,activation='sigmoid')

Final_model_VGC16 = tf.keras.Sequential([
  base_model_VGC16,
  Layer1,
  Layer2,
  Layer3,
  Layer4,
  Layer5,                                        
  Layer6,
  Layer7,
  Layer8,
  Layer9,
  prediction_layer
])
Final_model_VGC16.summary()



In [ ]:
from tensorflow import keras
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

def scheduler(epoch):
  if epoch < 10:
    return 0.001
  else:
    return 0.001 * tf.math.exp(0.1 * (10 - epoch))

callback1 = tf.keras.callbacks.LearningRateScheduler(scheduler)
callback2 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)


In [ ]:
Final_model_VGC16.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
Final_model_VGC16.summary()

In [ ]:

nb_epochs = 100
batch_size = 64
history = Final_model_VGC16.fit(
    norm_train_generator,
    steps_per_epoch = norm_train_generator.samples // batch_size,
    validation_data = norm_validation_generator, 
    validation_steps = norm_validation_generator.samples // batch_size,
    epochs = nb_epochs,
    callbacks = [callback1,callback2],
    )


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([0,1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                    target_size = (224,224),
                                                    batch_size = 1,
                                                    class_mode = 'binary')


In [ ]:
c = 0
for i in range(660):
  temp = Final_model_VGC16.predict(test_generator[i][0]) 
  if  temp > 0.5:
    temp = 1
  else:
    temp = 0
  if temp == test_generator[i][1].astype('int32')[0]:
    c+=1    
print(c)


In [ ]:
Final_model_VGC16.evaluate(test_generator,batch_size=1,steps=660)


In [ ]:
Final_model_VGC16.save('/content/drive/My Drive/Colab Notebooks/data/VGG16FineTunedwithoutSGS.h5') 

In [ ]:
base_model_VGC16 = tf.keras.applications.VGG16(input_shape=(224,224,3),
                                               include_top=False,
                                               weights=None,
                                               classes = 10
                                    )
base_model_VGC16.trainable = True

Layer1 = tf.keras.layers.Flatten()
Layer2 = tf.keras.layers.Dense(1024,activation='relu')
Layer3 = tf.keras.layers.Dropout(.6)
Layer4 = tf.keras.layers.Dense(128,activation='relu')
Layer5 = tf.keras.layers.Dropout(.5)
Layer6 = tf.keras.layers.Dense(64,activation='relu')
Layer7 = tf.keras.layers.Dropout(.5)
Layer8 = tf.keras.layers.Dense(8,activation='relu')
Layer9 = tf.keras.layers.Dropout(.2)
prediction_layer = tf.keras.layers.Dense(1,activation='sigmoid')

Final_model_VGC16 = tf.keras.Sequential([
  base_model_VGC16,
  Layer1,
  Layer2,
  Layer3,
  Layer4,
  Layer5,                                        
  Layer6,
  Layer7,
  Layer8,
  Layer9,
  prediction_layer
])
Final_model_VGC16.summary()


In [ ]:
Final_model_VGC16.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
Final_model_VGC16.summary()

In [ ]:

nb_epochs = 100
batch_size = 64
history = Final_model_VGC16.fit(
    norm_train_generator,
    steps_per_epoch = aug_train_generator.samples // batch_size,
    validation_data = aug_validation_generator, 
    validation_steps = aug_validation_generator.samples // batch_size,
    epochs = nb_epochs,
    callbacks = [callback1,callback2],
    )


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([0,1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
c = 0
for i in range(660):
  temp = Final_model_VGC16.predict(test_generator[i][0]) 
  if  temp > 0.5:
    temp = 1
  else:
    temp = 0
  if temp == test_generator[i][1].astype('int32')[0]:
    c+=1    
print(c)


In [ ]:
Final_model_VGC16.evaluate(test_generator,batch_size=1,steps=660)


In [ ]:
Final_model_VGc16.save('/content/drive/My Drive/Colab Notebooks/data/VGG16FineTunedwithSGS.h5') 